In [ ]:
import pandas as pd 
from bs4 import BeautifulSoup
from urllib.parse import quote
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# 전체 페이지에서 관광지명, 해시태그, 주소, 연락처, 상세정보 가져오기

In [ ]:
url = 'https://www.visitjeju.net/kr/detail/list?menuId=DOM_000001718000000000&cate1cd=cate0000000002#p1&pageSize=12&sortListType=reviewcnt&viewType=map&isShowBtag&tag'
driver = webdriver.Chrome()
driver.get(url)
time.sleep(2)
soup = BeautifulSoup(driver.page_source,'html.parser')

In [ ]:
#  해당 페이지를 리스트 형태로 보기 
driver.find_element(By.CLASS_NAME,'ico_sort.type_list').click()

In [ ]:
# 관광지명 가져오기
list_a = soup.select('p.s_tit')
list_a[0].text

In [ ]:
# 관광지명 클릭하기
driver.find_elements(By.CLASS_NAME,'s_tit')[0].click()

In [ ]:
#  상세페이지 url  가져오기
url_indi = driver.current_url

In [ ]:
# 해시태그 가져오기
tag_area=[]
for i in driver.find_elements(By.CLASS_NAME,'tag_area > p> a'):
    tag_area.append(i.text)
    print(i.text)

In [ ]:
# 주소 및 연락처,상세정보 가져오기
driver.find_element(By.CLASS_NAME, 'info_sub_cont')
juso =driver.find_elements(By.CLASS_NAME, 'info_sub_cont')[0].text
tel = driver.find_elements(By.CLASS_NAME, 'info_sub_cont')[1].text
info = driver.find_element(By.CLASS_NAME,'real').text

In [ ]:
# 원래 페이지로 돌아오기(두번 클릭)
driver.back()
driver.back()

In [ ]:
# 총 124페이지의 전체 페이지에서 관광지명가져오고 각 해당 관광지 상세페이지에서 해시태그, 주소, 연락처, 상세정보, url 가져오기
result = []
for page in range(1,125):
    url =f'https://www.visitjeju.net/kr/detail/list?menuId=DOM_000001718000000000&cate1cd=cate0000000002#p{page}&pageSize=12&sortListType=reviewcnt&viewType=map&isShowBtag&tag'
    time.sleep(2)
    driver = webdriver.Chrome()
    driver.get(url)
    time.sleep(5)
    soup = BeautifulSoup(driver.page_source,'html.parser')

    #  해당 페이지를 리스트 형태로 보기 
    driver.find_element(By.CLASS_NAME,'ico_sort.type_list').click()
    time.sleep(3)

    # 해당페이지에서 관광지명 가져오기
    list_a = soup.select('p.s_tit')

    # 관광지별 상세정보 새창에서 가져오기(해시태그, 주소, 연락처, 상세정보) 
    for i in range(len(list_a)):
        text = list_a[i].text

        # 상세페이지 클릭
        driver.find_elements(By.CLASS_NAME,'s_tit')[i].click()
        time.sleep(2)

        # 상세페이지 url 가져오기
        url_indi = driver.current_url
        time.sleep(2)

        # 해시태그
        tag_area=[]
        for i in driver.find_elements(By.CLASS_NAME,'tag_area > p> a'):
            tag_area.append(i.text)

        # 주소, 연락처, 상세정보
        juso =driver.find_elements(By.CLASS_NAME, 'info_sub_cont')[0].text
        tel = driver.find_elements(By.CLASS_NAME, 'info_sub_cont')[1].text
        info = driver.find_element(By.CLASS_NAME,'real').text
        result.append([text, juso,tel,info, tag_area,url_indi])

        # 원래 페이지로 돌아오기
        driver.back() 
        driver.back()

In [ ]:
# 데이터 저장하기
df = pd.DataFrame(result)
df.to_csv('./관광지상세정보.csv')